In [1]:
%pwd
!pip install requests


In [2]:
# Python variables
TR = 'tr_02_acl'
CC = 'cc_02_acl'

#TRAIN_PATH  = 'data/04-hyper/train.jsonl'
TRAIN_PATH  = 'data/02-acl-arc/train.jsonl'
#TRAIN_PATH  = 'data/07-imdb/train.jsonl'
TR_TSV  = 'emb/' + TR + '.tsv'
CC_TSV  = 'emb/' + CC + '.tsv'


### Filtering I (BM25 Retrieval)

In [3]:
import sys
import json
import requests
import time
start_time = time.time()
################################################################################
# Filtering
################################################################################
pretty = lambda x : json.dumps(x, indent=2, sort_keys=True)
solr_select = 'http://localhost:8983/solr/depcc-small/select?q='
with open(TRAIN_PATH, 'r') as train_file:
    json_lines = []
    lines = train_file.readlines()
    for line in lines:
        j = json.loads(line)
        json_lines.append(j)
N = len(json_lines)
print(N)


# Query with training example
j = json_lines[0]
query = j['text'].replace(' ', '+')
#query = ""
#with open('data/02-acl-arc/lda_union.txt', 'r') as file:
#    query = file.read()
print(query)
#print(len(query.split()))
#sys.exit()
rp_retrieval = requests.get(solr_select + query).json()
cc_docs = (rp_retrieval['response']['docs'])
print('Number of retrieved documents: %d' % len(cc_docs))
cc_doc0 = json.loads(cc_docs[0]['_src_'])
cc_doc100 = ""
for i in range(10):
    cc_doc100 += json.loads(cc_docs[i]['_src_'])['text']
#print(cc_doc100)
print("--- %s seconds ---" % (time.time() - start_time))
################################################################################


1688
Thus+,+over+the+past+few+years+,+along+with+advances+in+the+use+of+learning+and+statistical+methods+for+acquisition+of+full+parsers+(+Collins+,+1997+;+Charniak+,+1997a+;+Charniak+,+1997b+;+Ratnaparkhi+,+1997+)+,+significant+progress+has+been+made+on+the+use+of+statistical+learning+methods+to+recognize+shallow+parsing+patterns+syntactic+phrases+or+words+that+participate+in+a+syntactic+relationship+(+Church+,+1988+;+Ramshaw+and+Marcus+,+1995+;+Argamon+et+al.+,+1998+;+Cardie+and+Pierce+,+1998+;+Munoz+et+al.+,+1999+;+Punyakanok+and+Roth+,+2001+;+Buchholz+et+al.+,+1999+;+Tjong+Kim+Sang+and+Buchholz+,+2000+)+.
Number of retrieved documents: 10
--- 0.12327218055725098 seconds ---


### Segmentation by sentences(Documents -> Passages)

In [4]:
#!python -m spacy download en_core_web_sm
import spacy

nlp = spacy.load("en_core_web_sm", exclude=["parser"])
nlp.enable_pipe("senter")
doc = nlp(cc_doc100)
cc_psgs = []
psg = ''
num_tokens = 0
for sent in doc.sents:
    if num_tokens < 100:
        psg += sent.text
        num_tokens += len(sent)
    else:
#        print(num_tokens)
        cc_psgs.append({'doc_id' : '', 'doc_text'  : psg,  'title': ''  })
        num_tokens = 0
        psg = ''
#print(len(cc_psgs))

### Encoding Preparation

In [5]:
################################################################################
# Encoding Preparation
################################################################################

import csv
import subprocess

num_train = len(json_lines)
train_psgs = []
for i in range(num_train):
    train_dict = {'doc_id': str(i), 'doc_text': json_lines[i]['text'], 'title': ''}
    train_psgs.append(train_dict)
    
print(TR)
print(train_psgs[0]['doc_text'])
print()

with open(TR_TSV, 'w') as output_file:
    dw = csv.DictWriter(output_file, train_psgs[0].keys(), delimiter='\t')
    for tp in train_psgs:
        dw.writerow(tp)
        
        
with open(CC_TSV, 'w') as output_file:
    dw = csv.DictWriter(output_file, cc_psgs[0].keys(), delimiter='\t')
    for psg in cc_psgs:
        dw.writerow(psg)
print(CC)
print(cc_psgs[0]['doc_text'])  


tr_02_acl
Thus , over the past few years , along with advances in the use of learning and statistical methods for acquisition of full parsers ( Collins , 1997 ; Charniak , 1997a ; Charniak , 1997b ; Ratnaparkhi , 1997 ) , significant progress has been made on the use of statistical learning methods to recognize shallow parsing patterns syntactic phrases or words that participate in a syntactic relationship ( Church , 1988 ; Ramshaw and Marcus , 1995 ; Argamon et al. , 1998 ; Cardie and Pierce , 1998 ; Munoz et al. , 1999 ; Punyakanok and Roth , 2001 ; Buchholz et al. , 1999 ; Tjong Kim Sang and Buchholz , 2000 ) .

cc_02_acl
Over the past fifteen years there has been significant progress in the field of statistical parsing .Much of the work has focussed on supervised methods , where by ' ' supervised ' ' we mean that the training data consists of sentences and their associated syntactic trees ( for example , Charniak 1997 , Collins 1999 , Roark and Johnson 1999 ) .There are a number of

### Encode Train Data

In [6]:
# Encode TR (tsv-> embedding)


!bash emb/generate_embedding.sh tr_02_acl

Initialized host node040 as d.rank -1 on device=cuda, n_gpu=1, world size=1
16-bits training: False 
Reading saved model from /mnt/nfs/work1/696ds-s21/hhwang/DPR/data/checkpoint/retriever/multiset/bert-base-encoder.cp
model_state_dict keys odict_keys(['model_dict', 'optimizer_dict', 'scheduler_dict', 'offset', 'epoch', 'encoder_params'])
Overriding args parameter value from checkpoint state. Param = do_lower_case, value = True
Overriding args parameter value from checkpoint state. Param = pretrained_model_cfg, value = bert-base-uncased
Overriding args parameter value from checkpoint state. Param = encoder_model_type, value = hf_bert
Overriding args parameter value from checkpoint state. Param = sequence_length, value = 256
 **************** CONFIGURATION **************** 
batch_size                     -->   32
ctx_file                       -->   /mnt/nfs/work1/696ds-s21/hhwang/da/emb/tr_02_acl.tsv
device                         -->   cuda
distributed_world_size         -->   1
do_low

### Encode CC

In [7]:
# Encode CC (tsv->embedding)
!bash emb/generate_embedding.sh cc_02_acl

Initialized host node040 as d.rank -1 on device=cuda, n_gpu=1, world size=1
16-bits training: False 
Reading saved model from /mnt/nfs/work1/696ds-s21/hhwang/DPR/data/checkpoint/retriever/multiset/bert-base-encoder.cp
model_state_dict keys odict_keys(['model_dict', 'optimizer_dict', 'scheduler_dict', 'offset', 'epoch', 'encoder_params'])
Overriding args parameter value from checkpoint state. Param = do_lower_case, value = True
Overriding args parameter value from checkpoint state. Param = pretrained_model_cfg, value = bert-base-uncased
Overriding args parameter value from checkpoint state. Param = encoder_model_type, value = hf_bert
Overriding args parameter value from checkpoint state. Param = sequence_length, value = 256
 **************** CONFIGURATION **************** 
batch_size                     -->   32
ctx_file                       -->   /mnt/nfs/work1/696ds-s21/hhwang/da/emb/cc_02_acl.tsv
device                         -->   cuda
distributed_world_size         -->   1
do_low

In [8]:
MAX_TR_PSGS = len(train_psgs)
MAX_CC_PSGS = len(cc_psgs)
print(MAX_TR_PSGS)
print(MAX_CC_PSGS)

1688
416


### Nearest Neighbors

In [9]:
################################################################################
# Nearest Neighbor (FAISS)
################################################################################
#print(emb[0])
import numpy as np
import faiss

# Read CC embeddings (DATABASE)
cc_embeddings = np.load('emb/' + CC + '_0.pkl', allow_pickle=True)
print(cc_embeddings[0][1].shape)  # Dimension of the embedding
nb = len(cc_embeddings) # database size
d = cc_embeddings[0][1].size
print(nb,d)
xb = np.zeros((nb,d), dtype='float32')
for i in range(nb):
    xb[i] = cc_embeddings[i][1]
print('Number of CC passages: %d' % nb)

# Read train embeddings (QUERY)
train_embeddings = np.load('emb/' + TR + '_0.pkl', allow_pickle=True)
print(train_embeddings[0][1].shape)  # Dimension of the embedding
#nq = len(train_embeddings[:10]) # database size
nq = len(train_embeddings)
d = train_embeddings[0][1].size
print(nq,d)
xq = np.zeros((nq,d), dtype='float32')
for i in range(nq):
    xq[i] = train_embeddings[i][1]

print('Number of train passages: %d' % nq)
print(xq)


index = faiss.IndexFlatL2(d)   # build the index

print('trained? %r' % index.is_trained)
index.add(xb)                  # add vectors to the index
print('Total number of indexed CC passages: ', index.ntotal)
print()
print('Using an indentical CC set')
k = nb                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print('================================================================')
print('4 nearest neighbors')
print(I)
print()
print('distances(sanity check)')
print(D)
print()


print('===============================================================')
print('Using the query(train set)')
D, I = index.search(xq, k)     # actual search
print('4 nearest neighbors')
print(I[:5])                   # neighbors of the 5 first queries
print('\ndistances')
print(D)
print()



(768,)
416 768
Number of CC passages: 416
(768,)
1688 768
Number of train passages: 1688
[[-0.87804127  0.05203568 -0.06780454 ... -0.5197406  -0.2222211
  -0.02371504]
 [-0.7526125  -0.15401648  0.3375842  ... -0.70269537 -0.06798308
   0.0022837 ]
 [-0.43047038  0.07143795 -0.03356247 ... -0.5204008  -0.0784183
   0.17523418]
 ...
 [-0.08816212 -0.47372022  0.38190478 ... -0.2598227  -0.25476226
   0.04806864]
 [-0.58373874 -0.27933288  0.5223103  ... -0.47887757 -0.6835381
   0.09084422]
 [-0.4655094  -0.11000601  0.3970269  ... -0.4577696   0.09669067
  -0.4276332 ]]
trained? True
Total number of indexed CC passages:  416

Using an indentical CC set
4 nearest neighbors
[[  0   2 256 ... 303  47 320]
 [  1 205 113 ... 306 303 320]
 [  2   3   0 ... 303  47 320]
 [  3   2 321 ...  47 303 320]
 [  4   6   5 ... 299 320  47]]

distances(sanity check)
[[  0.        30.18142   31.350739 ... 106.24573  108.085365 116.75043 ]
 [  0.        36.323853  36.505314 ... 122.65997  130.44864  136

In [11]:
import textwrap
print('Train passage')
print(train_psgs[300]['doc_text'])
print()
print('CLOSEST passages in CC:')

for i in range(4):
    print('-------------------------------------------------------------')
    print('Closest %d' % i)
    closest = I[300][i]
    print(textwrap.fill(cc_psgs[closest]['doc_text'], 80))

print('------------------------------------------------------------')
print('...')

for i in range(MAX_CC_PSGS-4, MAX_CC_PSGS):
    print('-------------------------------------------------------------')
    print('Farthest %d' % i)
    closest = I[300][i]
    print(textwrap.fill(cc_psgs[closest]['doc_text'],80))


Train passage
It can be shown ( Berger et al. , 1996 ) that the use of this model with maximum likelihood parameter estimation is justified on information-theoretic grounds when q represents some prior knowledge about the true distribution and when the expected values of f in the training corpus are identical to their true expected values .3 There is no requirement that the components of f represent disjoint or statistically independent events .

CLOSEST passages in CC:
-------------------------------------------------------------
Closest 0
Although such generative models are easy to implement and are intuitive , it is
not always the case that generative models perform best , since they are
maximizing the joint probability of data and model , rather than directly
maximizing conditional probability .Because we do not have gold - standard
references for training a secondary conditional reranker , we incorporate weak
supervision of evaluations against the perceptual world during the proce